---
## **PROJET TAL** : Classification de morceaux de discours mélangés appartenants à 2 anciens présidents de la République française : François Mitterand ou Jacques Chirac
#### **Auteurs** : Mélodie FLEURY, Matéo PETITET 
#### **Matière** : UE TAL par Vincent GUIGUE - 3A IODAA AgroParisTech
#### **Date de rendue** : 12 février 2025
---

### Objectifs
Le but de ce projet est de déterminer à partir d'une liste de phrases si ces dernières ont été prononcées par François Mitterand ou par Jacques Chirac, deux anciens présidents de la République française. Dans ce notebook, différentes méthodes d'apprentissage vont être testées pour évaluer et comparer leur performance de classification.

### Import des librairies nécessaires

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import codecs
import re
import os
import os.path
import string
import time
import logging
import sys
import math
import random
import optuna
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from torch.utils.data import DataLoader, TensorDataset, Dataset

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch.nn as nn
from tqdm.autonotebook import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = "cpu"
print(device)

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

#Installation librairie ollama si pas installé
%pip install ollama
import ollama
from ollama import chat
from ollama import ChatResponse

### Les fonctions utilisées dans ce notebook

In [ ]:
def load_pres(fname):
    """
    Chargement des données en séparant les labels (variable "alllabs") des morceaux de discours des 2 anciens présidents (variable "alltxts") dans le fichier

    Args:
        fname (str): chemin du fichier des données d'entraînement et de test

    Returns: 
        alltxts (str): morceaux de discours des 2 anciens président du document
        alllabs (int): label associé à chaque morceau de discours = Si 0 alors la phrase provenait de Mr Mitterand sinon de Mr Chirac
    """
    alltxts = []
    alllabs = []
    s=codecs.open(fname, 'r','utf-8') # pour régler le codage
    while True:
        txt = s.readline()
        if(len(txt))<5:
            break
        #
        lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
        txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt).strip()
        if lab.count('M') >0:
            alllabs.append(0)
        else: 
            alllabs.append(1)
        alltxts.append(txt)
    return alltxts,alllabs

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

def preprocess(text):
    """
    Applique des pré-traitements sur un fichier textuel ("text"). Par exemple,
    la conversion des mots en minuscule, la suppression des nombres, de la
    ponctuation et des stopwords.

    Args: 
        text (str): Texte brute en entrée

    Returns: 
        text (str) : Texte pré-traité en sortie
    """
    # Conversion en minuscules
    text = text.lower().strip()
    # Suppression des nombres
    text = re.sub('[0-9]+', '', text)
    # Suppression des ponctuations
    punc = string.punctuation  
    punc += '\n\r\t'
    text = text.translate(str.maketrans(punc, ' ' * len(punc)))
    # Suppression des stopwords
    stop_words = set(stopwords.words('french'))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Rejoindre les tokens en une chaîne
    text = ' '.join(tokens)
    return text

class TextDataset(Dataset):
    """
    Classe permettant de structurer les morceaux de discours et les labels pour qu'ils soient plus facilement utilisées par un modèle Pytorch
    """
    def __init__(self, texts: list, labels):
        self.labels = labels
        self.phrasesnum = texts

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return self.phrasesnum[i], torch.tensor(self.labels[i])

def collate_fn(batch):
    """
    Personnalise le traitement des données sous forme de batch avant d'être entraîné dans un modèle

    Args:
        batch (tuple): Liste d'échantillon de morceaux de discours ("sequences") et de labels de l'ancien président associés

    Returns:
        padded_sequences (Tensor): Tensor des séquences de longueur uniforme après padding
        torch.tensor(lengths) (Tensor): Tensor contenant les longueurs originales des séquences
        torch.tensor(labels) (Tensor): Tensor contenant les étiquettes du batch
    """
    #Séparation des séquences et des labels sous la forme de tuples
    sequences, labels = zip(*batch)
    lengths = [len(seq) for seq in sequences]
    #permet de mettre toutes les séquences du batch à la même longueur en ajoutant des zéros
    padded_sequences = pad_sequence(sequences, batch_first=False)
    return padded_sequences, torch.tensor(lengths), torch.tensor(labels)

def generate_sinusoidal_embeddings(seq_len, d_model):
    """
    Génération des embeddings positionnels basés sur des sinusoïdes (sine et cosine) pour permettre au modèle de connaître l'ordre des tokens dans une séquence

    Args:
        seq_len (int): Longueur de la séquence
        d_model (int): Dimension de l'espace des embeddings

    Returns:
        : Embedding positionnel sinusoïdal
    """
    #Creation d'un tensor avec des entiers de 0 à seq_len -1
    position = torch.arange(seq_len).unsqueeze(1) #Ajout d'une dimension supplémentaire afin de créer une colonne de positions
    #Génération d'un tensor avec les valeurs appropriées à chaque dimension de d_model
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    #Tensor vide
    pe = torch.zeros(seq_len, d_model)
    #Application de la fonction sinus aux indices impairs des dimensions
    pe[:, 0::2] = torch.sin(position * div_term)
    #Application de la fonction cosinus aux indices pairs des dimensions
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe


def accuracy(yhat,y):
    """
    Calcul de la métrique d'évaluation des modèles avec le dataset de test

    Args:
        yhat (int): label prédit par le modèle
        y (int): label vrai

    Returns:
        Tensor: Accuracy soit la moyenne de bonnes prédictions
    """
    assert len(y.shape)==1 or y.size(1)==1
    return (torch.argmax(yhat,1).view(y.size(0),-1)== y.view(-1,1)).float().mean()

def train(model,epochs,train_loader,test_loader):
    """
    Entraînement du Transformers et récupération des valeurs d'accuracy pour le jeu d'entraînement et de test

    Args:
        model: modèle de Transformers
        epochs (int): nombre d'itération de descente de gradient
        train_loader: Jeu d'entraînement configuré par le DataLoader
        test_loader: Jeu de test configuré par le DataLoader

    Returns:
        list: Listes des valeurs d'accuracy du jeu de donnée d'entraînement et de test
    """
    optim = torch.optim.Adam(model.parameters(),lr=5e-4)    # choix optimizer
    model = model.to(device)
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss()                            # choix loss
    
    train_accuracies = []
    test_accuracies = []

    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x, lengths, y in train_loader:                            # boucle sur les batchs
            optim.zero_grad()
            x,y = x.to(device), y.to(device)                # y doit être un tensor (pas un int)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)                             # attention, il peut y avoir un batch + petit (le dernier)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)

        # Calcul de l'accuracy moyenne pour le jeu d'entraînement
        train_accuracy = round((cumacc / count).item(),2)
        train_accuracies.append(train_accuracy)
        
        if epoch % 2 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x, lengths, y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                
                # Calcul de l'accuracy moyenne pour le jeu de test
                test_accuracy = round((cumacc / count).item(),2)
                test_accuracies.append(test_accuracy)  # Ajout de l'accuracy test à la liste
                
    print(train_accuracies, test_accuracies)
    return train_accuracies, test_accuracies

#classification zero-shot
def classification_zero_shot(exemple):
    """
    Classification en mode "zero shot" utilisant le modèle Qwen

    Args:
        exemple (str): Phrase à associer au bon ancien président français qui l'a déjà énoncé

    Returns:
        str: Output du modèle Qwen
    """
    prompt = f"You are in a quizz. Here is a quote which has been prononced by an old french president, either Mitterand or Chirac. To win, you have to guess which of them said it sentence using only one word : 'Mitterand' or 'Chirac'. To win you can only say one word. Here is the quote :\n\n{exemple}"
    reponse = ollama.generate(model='qwen2.5:0.5b',prompt=prompt)
    return reponse

#classification few-shot
def classification_few_shot(exemple):
    """
    Classification en mode "few shot" utilisant le modèle Qwen

    Args:
        exemple (str): Phrase à associer au bon ancien président français qui l'a déjà énoncé

    Returns:
        str: Output du modèle Qwen
    """
    #Petit jeu d'entraînement pour le modèle
    txt = ["Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.",
           "A Brazzaville, que l'Afrique de demain se dessine.",
           "Moi je suis du côté de ceux qui pensent qu'il vaut mieux répartir plus justement l'effort et le produit de la nation.",
           "Songez que la femme mariée a eu beaucoup de peine à acquérir un statut juridique conforme à l'idée que nous nous faisons des rapports entre les femmes et les hommes."]
    
    prompt = f"These texts are in the category 'Chirac': \n{txt[0]}\n and \n{txt[1]}.\n\nThese texts are in the category 'Mitterand': \n{txt[2]}\n and \n{txt[-1]}.\n\n Give the category of the following text in only one word: 'Chirac' or 'Mitterand' :\n\n{exemple}"
    reponse = ollama.generate(model='qwen2.5:0.5b',prompt=prompt)
    return reponse

#évaluation des performances de Qwen
def test_perf(dataset,methode,labels_vrais):
    """Évaluation de la bonne classification de chaque phrase à l'ancien président de la République qui l'avait énoncé via l'usage du modèle Qwen

    Args:
        dataset (str): Liste de phrases énoncés soit par Mr François Mitterand soit par Mr Jacques Chirac
        methode (str): choix de la méthode "zero shot" ou "few shot"
        labels_vrais (int): liste des labels rééls à prédire associé aux anciens président français

    Returns:
       int: nombre de phrases bien classées, mal classées ou non classées par le modèle
    """
    # Shuffle two lists with same order
    temp = list(zip(dataset, labels_vrais))
    random.shuffle(temp)
    dataset, labels_vrais = zip(*temp)
    # res1 and res2 come out as tuples, and so must be converted to lists.
    dataset, labels_vrais = list(dataset), list(labels_vrais)

    ok, pb_rep, nul = 0, 0, 0

    pattern_mitterand = re.compile(r"(?i)\s*mitterand\s*[\W]*")
    pattern_chirac = re.compile(r"(?i)\s*chirac\s*[\W]*")
    #(?i) : Active l'option d'insensibilité à la casse (case insensitive), ce qui signifie que "Mitterand" et "mitterand" seront tous deux reconnus.
    #\s* : Correspond à zéro ou plusieurs espaces avant et après le mot "Mitterand".
    #\W : correspond à tout caractère non alphanumérique.

    for k in range(len(dataset)):
        response = methode(dataset[k]).response
        if (pattern_mitterand.match(response) and labels_vrais[k] == 0) or (pattern_chirac.match(response) and labels_vrais[k] == 1):
            ok += 1
        elif not pattern_mitterand.match(response) and not pattern_chirac.match(response):
            pb_rep += 1
        else:
            nul += 1

    return ok, pb_rep, nul

### CHARGEMENT DES JEUX DE DONNÉES

In [ ]:
trainCorpus = "./ressources/AFDpresidentutf8/corpus.tache1.learn.utf8"
testCorpus = "./ressources/AFDpresidentutf8/corpus.tache1.test.utf8"

alltxts_train, alllabs_train = load_pres(trainCorpus)
alltxts_test, alllabs_test = load_pres(testCorpus)

In [ ]:
#Affichage des données brutes
print(len(alltxts_train),len(alllabs_train))
print(alltxts_train[:3])
print(alllabs_train[:3])
print(alltxts_train[-1])
print(alllabs_train[-1])

In [ ]:
#permet de compter l'effectif de chaque valeur unique dans une liste
counter_train = Counter(alllabs_train)
counter_test = Counter(alllabs_test)

print("Nombre de phrases du corpus d'entraînement : ", len(alltxts_train))
print("----> # de l'ancien président français Jacques Chirac (label '1') : ", counter_train[1])
print("----> # de l'ancien président français François Mitterand (label '0') : ", counter_train[0])

print("Nombre de phrases du corpus de test : ", len(alllabs_test))
print("----> # de l'ancien président français Jacques Chirac (label '1') : ", counter_test[1])
print("----> # de l'ancien président français François Mitterand (label '0') : ", counter_test[0])

On peut remarquer que les **classes sont assez déséquilibrées**. En effet, dans le corpus d'entraînement, seulement 13% des phrases ont été prononcées par François Mitterand et 10% pour le corpus de test. Cela peut avoir amener des algorithmes à mieux identifier les phrases de Mr Chirac que ceux de Mr Mitterand.

### PRE-PROCESSING DES DONNÉES BRUTES

In [ ]:
#Application du preprocessing à mon texte d'apprentissage et de test
alltxts_train_prep = []
alltxts_test_prep = []

for train_sentence in alltxts_train:
    alltxt_prep = preprocess(train_sentence)
    alltxts_train_prep += [alltxt_prep]

for test_sentence in alltxts_test:
    alltxt_prep = preprocess(test_sentence)
    alltxts_test_prep += [alltxt_prep]

In [ ]:
#10 premières lignes du corpus d'entraînement pré-traité
alltxts_train_prep[:10]

### Première partie : BAG OF WORDS
<span style="color:black"> Les <b> bags of words (BoW) </b> sont des représentations textuelles utilisées pour transformer des textes en vecteurs numériques, où chaque dimension correspond à un mot unique dans un vocabulaire, et la valeur indique le nombre d'occurences de ce mot dans le texte. Cette méthode simple est largement utilisée pour de la classification de texte comme c'est le cas ici. Cependant, cette technique a quelques limites notamment celle <u> de ne pas prendre en compte la position des mots ni les relations sémantiques entre eux </u>. </span>

In [ ]:
#Permet de transformer chaque ligne en embedding type "one-hot-encoding"
vectorizer = CountVectorizer()

In [ ]:
X_prep = vectorizer.fit_transform(alltxts_train_prep)
X_noprep = vectorizer.fit_transform(alltxts_train)
print(vectorizer.get_feature_names_out()) #tous les mots différents du texte

In [ ]:
#Naïve Bayes
nb_clf = MultinomialNB()
fit_nb_prep = nb_clf.fit(X_prep, alllabs_train)
fit_nb_noprep = nb_clf.fit(X_noprep, alllabs_train)

#Logistic Regression
lr_clf = LogisticRegression(random_state=0, solver='lbfgs',n_jobs=-1)
fit_lr_prep = lr_clf.fit(X_prep, alllabs_train)
fit_lr_noprep = lr_clf.fit(X_noprep, alllabs_train)

#Linear SVM
svm_clf = LinearSVC(random_state=0, tol=1e-5)
fit_svm_prep = svm_clf.fit(X_prep, alllabs_train)
fit_svm_noprep = svm_clf.fit(X_noprep, alllabs_train)

#Labels vrais
true = alllabs_test

#Morceaux de discours avec application des fonctions de préprocessing puis sans
test_corpus_prep = alltxts_test_prep
test_corpus_noprep = alltxts_test

#Application du one-hot-encoding à chaque morceau de discours
X_test_prep = vectorizer.transform(test_corpus_prep)
X_test_noprep = vectorizer.transform(test_corpus_noprep)

#Prédictions des labels
pred_nb_prep = nb_clf.predict(X_test_prep)
pred_nb_noprep = nb_clf.predict(X_test_noprep)

pred_lr_prep = lr_clf.predict(X_test_prep)
pred_lr_noprep = lr_clf.predict(X_test_noprep)

pred_svm_prep = svm_clf.predict(X_test_prep)
pred_svm_noprep = svm_clf.predict(X_test_noprep)

#Calcul de l'accuracy pour chaque modèle avec et sans préprocessing
acc_nb_prep = accuracy_score(true, pred_nb_prep)
acc_nb_noprep = accuracy_score(true, pred_nb_noprep)

acc_lr_prep = accuracy_score(true, pred_lr_prep)
acc_lr_noprep = accuracy_score(true, pred_lr_noprep)

acc_svm_prep = accuracy_score(true, pred_svm_prep)
acc_svm_noprep = accuracy_score(true, pred_svm_noprep)

#Affichage des accuracy pour chaque modèle avec et sans préprocessing
print(f"Naïve Bayes accuracy with preprocessing: {acc_nb_prep}")
print(f"Naïve Bayes accuracy without preprocessing: {acc_nb_noprep}")

print(f"Logistic Regression accuracy with preprocessing: {acc_lr_prep}")
print(f"Logistic Regression accuracy without preprocessing: {acc_lr_noprep}")

print(f"SVM accuracy with preprocessing: {acc_svm_prep}")
print(f"SVM accuracy without preprocessing: {acc_svm_noprep}")

In [ ]:
#Représentation graphique des accuracy avec pré-processing et sans preprocessing
fig, ax = plt.subplots()

X = ['Naïve Bayes', 'Logistic Regression', 'SVM']

prep = [acc_nb_prep, acc_lr_prep, acc_svm_prep]
no_prep = [acc_nb_noprep, acc_lr_noprep, acc_svm_noprep]

X_axis = np.arange(len(X)) 
  
plt.bar(X_axis - 0.2, prep, 0.4, label = 'Avec pré-processing') 
plt.bar(X_axis + 0.2, no_prep, 0.4, label = 'Sans pré-processing') 

plt.xticks(X_axis, X) 
plt.legend() 
ax.set_ylabel('Accuracy')
ax.set_title("Taux de bonne classification pour chaque algorithme \n selon si pré-processing des données ou non", pad = 10)
ax.set_ylim(0.5,1)

#plt.show() #Affichage commenté pour fixer les derniers résultats

**Image originale :**
![](images/barplot_preprocessing_or_not.png)

<span style="color:black"> <u> Conclusion : </u> On remarque que le taux de bonne classification des morceaux de discours des 2 anciens présidents de la république est bon. En effet, on obtient des valeurs d'accuracy compris entre entre 0.78 et 0.85. Cependant, on remarque que la régression logistique a de légères meilleures performances que ce soit avec ou sans le pré-processing des données brutes. Concernant, ce pré-processing, on voit que pour les 3 algorithmes utilisés, le pré-processing améliore subtilement les résultats avec une augmentation de 0.03 points en moyenne.</span>

### 2ème partie : Usage d'un modèle de Transformers
<span style="color:black"> Les <b> Transformers </b> sont des modèles d'apprentissage profond utilisés principalement pour le traitement du langage naturel. Ces types de modèles utilisent un <u> mécanisme d'attention </u> pour traiter simultanément toutes les parties d'une séquence. Cela permet d'analyser efficacement des relations à longue distance dans les données. Ainsi, les Transformers sont particulièrement performants pour des tâches comme de la classification de texte. </span>

In [ ]:
# Initialiser un tokenizer WordPiece
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
#taille du vocabulaire
VOC_SIZE = 10000
#taille des batch
BATCH_SIZE = 32
MAX_CHAR_SIZE = 1000
NB_DOC_MAX = 12500 # par classe

# Définir les pré-traitements
tokenizer.pre_tokenizer = Whitespace()

# Créer un entraîneur avec un vocabulaire cible
trainer = WordPieceTrainer(
    vocab_size=10000,  # Limite du vocabulaire
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Entraîner le tokenizer sur vos données
tokenizer.train_from_iterator(alltxts_train_prep, trainer=trainer)

In [ ]:
PAD = tokenizer.encode("[PAD]").ids[0]
print("PAD",PAD)

In [ ]:
# fabrication de tous les codes
allcodes = [torch.tensor(tokenizer.encode("[CLS] " + p).ids) for p in alltxts_train_prep]
allcodes_test = [torch.tensor(tokenizer.encode("[CLS] " + p).ids) for p in alltxts_test_prep]

In [ ]:
ds_train = TextDataset(allcodes,alllabs_train)
ds_test  = TextDataset(allcodes_test,alllabs_test)

In [ ]:
#encapsulation des tokens dans un format DataLoader
train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [ ]:
#modèle de Transformers
class TransSent(nn.Module):
    """
    Modèle Transformer pour la classification de texte

    Args:
        nn.Module (module): module "Module" de Pytorch
    """

    def __init__(self, emb_size, voc_size, num_layers, num_heads, hidden_size_mlp, output_size, maxlen=1000):
        super(TransSent, self).__init__()

        self.emb_size = emb_size

        # 1. Embedding de chaque mot
        self.emb = nn.Embedding(voc_size, emb_size)
        
        # 2. Transformer Encoder Layer
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_size, 
            nhead=num_heads, 
            dim_feedforward=hidden_size_mlp,
            activation='relu'
        )

        # 3. Transformer Encoder
        self.trans = nn.TransformerEncoder(
            self.encoder_layer, 
            num_layers=num_layers
        )

        # 4. Ajout des embeddings positionnels (sinusoïdaux)
        self.register_buffer("posemb", generate_sinusoidal_embeddings(maxlen, self.emb_size).unsqueeze(1))

        # 5. Couche de classification (depuis le vecteur [CLS])
        self.h2o = nn.Linear(emb_size, output_size)


    def forward(self, input, lengths=None):
        maxlen = input.size(0)

        padding_mask = (input[:, :] == PAD).T 

        # 1. translation of the input from int to emb + ajout des positional embeddings
        xemb = self.emb(input) 
        xemb += self.posemb[:maxlen,:,:]
        
        # 2. Passage dans le transformer... Avec le masque pour le padding
        encoded_output = self.trans(xemb, src_key_padding_mask=padding_mask)
        
        # 3. Appliquer la classification sur le CLS
        output = self.h2o(encoded_output[0,:,:]).squeeze(0)
        
        return output

In [ ]:
# choose hidden size
emb_size = 128
voc_size = 10000
num_layers = 1
num_heads = 1
hidden_size_mlp = 128
output_size = 2
# build network
transf = TransSent( emb_size, voc_size, num_layers, num_heads, hidden_size_mlp , output_size)
transf.name = "TransSent-"+time.asctime()

In [ ]:
#training du transformers
n_epoch = 15
train_list, test_list = train(transf, n_epoch, train_loader, test_loader)

In [ ]:
# À la fin de l'entraînement, on trace le graphique de l'accuracy
plt.plot(range(n_epoch), train_list, label='Train Accuracy')
plt.plot(range(0, n_epoch, 2), test_list, label='Test Accuracy')  # Afficher l'accuracy de test tous les 2 epochs
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Évolution des performances de classification \n du transformers au cours des époques')
plt.legend()
plt.show()

**Image originale :**
![](images/perf_transformers.png)

<span style="color:black"> <u> Conclusion : </u> On remarque que le taux de bonne classification des morceaux de discours des 2 anciens présidents de la république en utilisant un transformers est bon aussi. En effet, on a au minimum 80% des discours qui ont été bien classés en appliquant le pré-processing sur les données brutes. Cependant, on remarque un grand écart entre l'accuracy du jeu d'entraînement et du jeu de test. On pourrait donc émettre l'hypothèse d'un surapprentissage du modèle. Pour éviter le surapprentissage, il est possible de faire de la régularisation en appliquant la technique dropout ou en augmentant les poids des observations mal classées.</span>

### 3ème partie : Ollama

<span style="color:black"> Ollama est une plateforme permettant de télécharger et exécuter localement des modèles de langage open-source optimisé. Parmi les modèles proposés, il y a Qwen qui est réputé pour être un bon équilibre entre la performance et le poids. Pour évaluer la performance du modèle de langage Qwen sur cette tâche, on va utiliser 2 méthodes différentes appelées "zero shot" et "few shot". Le premier consiste à réaliser de la classification sans exemple et le second, de la classification avec quelques exemples. On va analyser s'il y a des différences de performances ou pas à l'aide de la fonction "test_perf". Cette dernière va séparer les réponses du prompt selon si elles sont bien classifiées (réponse et bon label associé), selon si elles ont un problème de format de réponse ("cette phrase a été prononcée à par Chirac" au lieu de "Chirac" seulement), enfin, selon si elles sont mal classifées (réponse et mauvais label associé). </span>

In [ ]:
#Lancement de la classification du modèle Qwen en mode "zero shot" et "few shot"
rep_0shot = classification_zero_shot(alltxts_train[115])
print(alllabs_train[115])
print(rep_0shot.response)

rep_fewshot = classification_few_shot(alltxts_train[115])
print(alltxts_train[115])
print(rep_fewshot.response)

In [ ]:
n = 200 #seulement un échantillon est sélectionné pour accélerer le temps de traitement

#Taux d'accuracy classification zero shot modèle ollama
o1,p1,n1 = test_perf(alltxts_test_prep[:n], classification_zero_shot, alllabs_test[:n])
print(o1,p1,n1)

perc_o1 = 100*o1/(len(alltxts_test_prep[:n]))
perc_p1 = 100*p1/(len(alltxts_test_prep[:n]))
perc_n1 = 100*n1/(len(alltxts_test_prep[:n]))

print(
    'Zero-shot\n ok : ', perc_o1, '%', 
    'Nuls : ', perc_n1, '%', 
    'Mauvais format de réponse : ', perc_p1, '%.')

#Taux d'accuracy classification few shot modèle ollama
o2,p2,n2 = test_perf(alltxts_test_prep[:n], classification_few_shot, alllabs_test[:n])

perc_o2 = 100*o2/(len(alltxts_test_prep[:n]))
perc_p2 = 100*p2/(len(alltxts_test_prep[:n]))
perc_n2 = 100*n2/(len(alltxts_test_prep[:n]))

print(
    'Few-shot\n ok : ', perc_o2, '%', 
    'nuls : ', perc_n2, '%', 
    'Mauvais format de réponse : ', perc_p2, '%.')

In [ ]:
#Représentation graphique des accuracy avec la méthode zero shot et few shot
fig, ax = plt.subplots()

X = ['Bonne \n classification', 'Mauvais format \n de réponse', 'Nuls']

zero_shot = [perc_o1, perc_p1, perc_n1]
few_shot = [perc_o2, perc_p2, perc_n2]

X_axis = np.arange(len(X)) 
  
plt.bar(X_axis - 0.2, zero_shot, 0.4, label = 'zero shot') 
plt.bar(X_axis + 0.2, few_shot, 0.4, label = 'few shot') 

plt.xticks(X_axis, X) 
plt.legend() 
ax.set_ylabel('Accuracy')
ax.set_title(
    "Taux de bonne classification, de bonne classification mais mauvais format de réponse \n et de réponses nulles du modèle Qwen selon l'usage de la méthode 'zero shot' ou 'few shot'", 
    pad = 10)

plt.show()

**Image originale :**
![](images/ollama_discours.png)

<span style="color:black"> <u> Conclusion : </u> On remarque que le taux de bonne classification des morceaux de discours des 2 anciens présidents de la république en utilisant le modèle Qwen d'Ollama est drastiquement plus élevé en donnant quelques exemples au modèle (méthode "few-shot") que sans (méthode "one-shot"). En effet, on passe d'un score de 30% environ à quasiment 60% de phrases associées à son bon auteur. Également, le taux de mauvais format de réponse diminue en utilisant la méthode "few-shot" en passant de 20% de mauvaise classification avec la méthode "zero-shot" à 10% environ. Concernant les phrases mal classées par le modèle, on passe de plus de 50% avec la méthode zero-shot (elle est par ailleurs la classe majoritaire avec cette dernière méthode) à 30% environ. 
Finalement, le taux de discours bien classés reste majoritaires lorsque le modèle de language reçoit quelques exemples d'entraînement, contrairement à la méthode "zero-shot" où il y a 1 phrases sur 2 mal classée environ.</span>

### Conclusion générale

<span style="color:black">Au final, l'exercice de classer correctement des phrases provenant de discours de 2 anciens présidents de la République française, à partir de différentes méthodes d'apprentissages supervisées ou non, montrent plusieurs choses.
- D'abord, les méthodes bag-of-words et transformers montrent de bonnes performances de classification des discours et plutôt équivalentes même s'il y a de l'over-fitting apparent pour le modèle transformers. Il pourrait donc avoir le potentiel de faire mieux. 
- De plus, on s'est rendu compte que le pré-traitement des données brutes est essentiel pour le gain de performances de classification. 
- Enfin, concernant le modèle Qwen d'Ollama, les méthodes "zero-shot" et "few-shot" nous enseignent que Qwen améliore considérablement ses performances avec l'entraînement au préalable sur quelques exemples (méthode "few-shots"). La prise en compte de certaines erreurs d'orthographe ou de majuscules avec une expression régulière a également aidé à améliorer la précision de classification du modèle. </span>